In [1]:
#Notebook to visualize the macro function (which is responsible to detect if the trend is globally bear or bull)
import sys
sys.path.append("..")

from core import indicators as ic
from core import strat
from core.common import VBTfunc
import vectorbtpro as vbt
from vectorbtpro.utils.config import Config
import numpy as np
import gc
import talib
import pandas as pd
import math
import copy
from numba import njit

from core.macro import VBTMACROTREND, VBTMACROVIS

In [8]:
class Vis(VBTfunc):
    def __init__(self,input_ust):
        self.close=input_ust.close
        self.macro_trend=VBTMACROTREND.run(self.close).macro_trend

    def macro_mode(self,mode_to_vis):
        t=VBTMACROVIS.run(self.macro_trend,mode_to_vis=mode_to_vis)
        self.ents=t.entries
        self.exs=t.exits
    
    def manual_calculate_pf(self,mode_to_vis):
        self.macro_mode(mode_to_vis)
        pf=vbt.Portfolio.from_signals(self.close, self.ents,self.exs,freq="1d")
        return pf

In [9]:
period="2007_2022_08"
symbol_index="CAC40"
ust=strat.StratHold(period,symbol_index=symbol_index) #it_is_index

  0%|          | 0/39 [00:00<?, ?it/s]

In [10]:
o=Vis(ust) #,index=True

In [11]:
pf=o.manual_calculate_pf(0) #-1=Bull, 0=Uncertain, 1=Bear
pf.returns_stats()

/home/maxime/.local/lib/python3.8/site-packages/vectorbtpro/portfolio/base.py:7085: UserWarning: Object has multiple columns. Aggregated some metrics using <function mean at 0x7f78d024a040>. Pass column to select a single column/group.
  return getattr(returns_acc, "stats")(**kwargs)


Start                           2007-01-02 00:00:00+00:00
End                             2022-08-31 00:00:00+00:00
Period                                 4005 days 00:00:00
Total Return [%]                                35.938061
Benchmark Return [%]                           305.888611
Annualized Return [%]                             0.82292
Annualized Volatility [%]                       19.778256
Max Drawdown [%]                                47.474893
Max Drawdown Duration        2520 days 20:55:23.076923072
Sharpe Ratio                                     0.154699
Calmar Ratio                                     0.041181
Omega Ratio                                       1.04366
Sortino Ratio                                    0.238582
Skew                                             0.267621
Kurtosis                                        30.218265
Tail Ratio                                       1.043571
Common Sense Ratio                               1.055124
Value at Risk 

In [12]:
#pf.plot()
pf.plot(column=pf.wrapper.columns[3])

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '349b9386-adce-47df-b623-416ac44cf7ce',
              'x': array([datetime.datetime(2007, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 3, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2007, 1, 4, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2022, 8, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 8, 30, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 8, 31, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([37.9964447 , 38.47508621, 36.59735107, ..., 20.64999962,